In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from __future__ import print_function
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot as plt, gridspec
from plotutils import *


In [ ]:
basedir = '../benchmarks_py/sfi_scaling006'

DO_SAVE            = True # whether to save pdf output


In [ ]:
SCALING_TYPES = ['agents','memes','lexes','lexesmemes','lexesmemesagents']
DO_RESCALE_X = False

for scalingtype in SCALING_TYPES:
#scalingtype = 'agents'
    COL, desc, data, mutators = get_data(basedir, scalingtype)

    if DO_RESCALE_X:
        mutator_scaling_slopes = {}
        comprehension_cutoff_loc = {}

        for mndx, m in enumerate(sorted(mutators)):
            _, xs, ys, _ = get_scaling_vals(data, COL, m, 'Comprehension')
            slope,_ = plot_scatter(xs, ys, loglog=True, regression_line=True, do_plot=False)
            mutator_scaling_slopes[m] = slope
            comprehension_cutoff_loc[m] = (ys/xs**slope).mean()


    PLOT_COLS = ['Comprehension', 'AcceptanceRate', 'MeanGrammarDrift', 'AgentGrammarDrift', 'GrammarVar']
    fig=plt.figure(figsize=(12,15))
    for pndx, pCol in enumerate(PLOT_COLS):
        print("Doing %s" % pCol)
        #plt.suptitle('Scaling with %s'%desc, fontsize=17, y=1.01)

        gs = gridspec.GridSpec(len(PLOT_COLS),len(mutators))
        for mndx, m in enumerate(sorted(mutators)):
            plot_dfs, xs, ys, ylabel = get_scaling_vals(data, COL, m, pCol)

            plt.subplot(gs[pndx,mndx])
            plt.semilogx()

            ylabel = pCol
            xlabel = 'Steps'
            for cfg, df in plot_dfs:
                v = float(cfg[COL])
                vals = df[pCol]
                if DO_RESCALE_X:
                    xlabel = '$Steps \\times %s^{%0.2f}$' % (desc.replace('#', '\\#'), -mutator_scaling_slopes[m])
                    rescaled = df.Step/(v**mutator_scaling_slopes[m])
                    show_line = True
                    if pCol.endswith('Drift') or pCol.endswith('GrammarVar'):
                        if len(xs):
                            slope, _ = plot_scatter(xs, ys, loglog=True, regression_line=True, do_plot=False)
                        ylabel = '$%s \\times %s^{%0.2f}$' % (COL_NAMES.get(pCol, pCol), desc.replace('#', '\\#'), -slope)
                        vals = vals/(v**slope)
                    l=plt.plot(rescaled, vals, label=cfg[COL])
                    if not show_line: l[0].set_alpha(0.0)
                    if v==21:
                        plt.xlim([0, 2*np.max(rescaled)])
                    plt.vlines(comprehension_cutoff_loc[m], plt.ylim()[0], plt.ylim()[1], linestyle=':')
                else:
                    l=plt.plot(df.Step, vals, label=cfg[COL])

            plt.xticks([])
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)

            cTitle = 'Measure: %s' % COL_NAMES.get(pCol, pCol)
            if pndx == 0:
                cTitle = 'Mutator: %s\n'%m + cTitle
            plt.title(cTitle)
            if pndx == 0 and mndx == 0 and DO_RESCALE_X:
                leg = plt.legend(title=desc)
                for lh in leg.legendHandles: 
                    lh.set_alpha(1)

            #break

        plt.tight_layout()
    if not DO_RESCALE_X:
        handles, labels = plt.gca().get_legend_handles_labels()
        leg=fig.legend(handles, labels, loc='upper center', ncol=len(handles), fontsize=12, 
                       bbox_to_anchor=(0., 0.95, 1., .102), title=desc)
        leg.get_title().set_fontsize('14') #legend 'Title' fontsize


    if DO_SAVE:
        plt.savefig('output/trajectories_2019_%s%s.pdf' % (scalingtype, '_rescaled' if DO_RESCALE_X else ''), bbox_inches='tight')
    